In [1]:
import time
import gymnasium
from miniwob.action import ActionTypes
from dom_processing import iterate_dom, dict2html, prepare_t5_input
from inference_points import infer_remote_model, parse_t5_output_action
from cc_net5_tokenizer import CCNeT5Tokenizer
from cosine_similarity import find_closest_embeddings
import torch
import numpy as np
import json
task_name = 'click-test-2-v1'
task_name = 'email-inbox'
env = gymnasium.make(f"miniwob/{task_name}")


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/gymnasium/envs/registration.py:521: UserWarning: WARN: Overriding environment miniwob/bisect-angle-v1 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/gymnasium/envs/registration.py:521: UserWarning: WARN: Overriding environment miniwob/book-flight-v1 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/gymnasium/envs/registration.py:521: UserWarning: WARN: Overriding environment miniwob/book-flight-nodelay-v1 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/gymnasium/envs/registration.py:521: UserWarning: WARN: Overriding environment miniwob

In [2]:
def convert_to_nested_dict(elements):
    # Create a dictionary to store elements by their 'ref' value
    element_dict = {element['ref']: element for element in elements}

    # Iterate through the elements to build the nested structure
    for element in elements:
        parent_ref = element['parent']
        if parent_ref != 0:
            parent_element = element_dict.get(parent_ref)
            if 'children' not in parent_element:
                parent_element['children'] = []
            if 'children' not in element:
                # Empty children field
                element['children'] = []
            parent_element['children'].append(element)

    # Find the root element (element with 'parent' = 0)
    root_element = None
    for element in elements:
        if element['parent'] == 0:
            root_element = element
            break

    return root_element

# format image (3, 210, 160) for request by normalizing it
def format_image(image):
    rgb_data = np.transpose(np.array(image) / 255.0, (2, 0, 1))
    return rgb_data

def format_language_rgb_input(rgb_data, dom_data, utterance_data, task_data):
    # Put language task together
    language_input = np.concatenate((dom_data, utterance_data, task_data))
    #language_input = dom_data
    print(language_input.shape)

    rgb_data = torch.from_numpy(rgb_data).type(torch.float32)
    language_input = torch.from_numpy(language_input).type(torch.long)
    language_input = tokenizer.embedding_fn(language_input) # Create embeddings for language
    print(f'rgb_data: {rgb_data.shape}, language_input: {language_input.shape}')
    return rgb_data, language_input

import json

def prepare_ccnet5_payload(rgb_data, language_input, previous_action, t5_output_tensor):
    cc_net5_payload = {
        'rgb_input': rgb_data,
        'language_input': language_input,
        'previous_action': previous_action,
        't5_output': t5_output_tensor,
    }

    cc_net5_payload_numpy = {
        key: value.detach().numpy().tolist() if isinstance(value, torch.Tensor) else value
        for key, value in cc_net5_payload.items()
    }

    json_data = json.dumps(cc_net5_payload_numpy)
    return json_data


# Update the mapping dictionary of ref with page changes
def update_ref2_random(old_dict, old_rand2ref, new_dict):
    _remvoved_refs_keys = old_dict.keys() - new_dict.keys()
    added_refs_keys = new_dict.keys() - old_dict.keys() # no use of old refs, better to keep them in old_dict

    # create subset_dictionary
    added_refs = {}
    for added_ref in added_refs_keys:
        if added_ref in old_dict.keys():
            added_refs[added_ref] = old_dict[added_ref]
        else:
            added_refs[added_ref] = new_dict[added_ref]

    # attribute random to new refs:
    added_refs_dict, added_rand2refs = randomize_ref_dict(list(added_refs.keys()))

    # Update the old dictionary with the new mapping
    old_dict.update(added_refs_dict)
    old_rand2ref.update(added_rand2refs)

    # These are the new mappings
    return old_dict, old_rand2ref


In [3]:
# Prepare our Embedding Function
embedding_fn_path = '/Users/lucas-andreithil/PycharmProjects/miniwob-plusplus/embedding_weights.pth'
vocab_path="vocab.txt"
tokenizer = CCNeT5Tokenizer(vocab_path=vocab_path, embedding_fn_path=embedding_fn_path)

t5_url = 'https://5350-34-170-164-189.ngrok-free.app'
cc_net5_url = 'https://e63b-34-90-129-210.ngrok-free.app'

pad: 1591
Loaded CC_NeT5 Tokenizer with vocabulary size being 1592.


In [18]:
# Steps with TODOS

# Wrap the code in try-finally to ensure proper cleanup.
try:
  action_history = []
  previous_action = torch.zeros(577) # previous action default values

  # Start a new episode.
  observation, info = env.reset()
  assert observation["utterance"] == "Click button ONE."
  assert observation["fields"] == [("target", "ONE")]

  # 1: Reformat Screenshot Shape
  screenshot = observation['screenshot']
  print(f'screenshot shape: {screenshot.shape}')

  # 2: Check Utterance
  utterance = observation['utterance']

  # 3: Process DOM
  nested_dict = convert_to_nested_dict(observation['dom_elements'])
  target_ref, keydown_text, new_dom = iterate_dom(nested_dict)
  # Eventually randomize the refs

  # 4: Model call
  # a. T5 call
  # Format for T5 call
  # todo: keep action history as well
  html_str = dict2html(new_dom)
  t5_input = prepare_t5_input(action_history, utterance, html_str)
  print(f't5_input: {t5_input}')
  t5_output = infer_remote_model(t5_input, t5_url)
  t5_action, t5_ref, t5_keydown = parse_t5_output_action(t5_output)
  print(f't5_action: {t5_action}, t5_ref: {t5_ref}, t5_keydown: {t5_keydown}')

  # format tensor T5_output into Embeddings (flattened tensor)
  t5_output_tensor = torch.zeros(577)
  if t5_action == 'keydown':
      t5_output_tensor[0:1] = 1
  #t_ref = torch.from_numpy(np.array(tokenizer.truncate_pad_entry(tokenizer.tokenize_string(t5_ref), 64)))
  t_ref = torch.flatten(tokenizer.embedding_fn(torch.from_numpy(np.array(tokenizer.stoi[str(t5_ref)])))).detach()
  t_keydown = torch.flatten(tokenizer.embedding_fn(torch.from_numpy(np.array(tokenizer.truncate_pad_entry(tokenizer.tokenize_string(str(t5_keydown)), 8))))).detach()
  t5_output_tensor[1:65] = t_ref
  t5_output_tensor[65:577] = t_keydown

  # b. CC-NeT5 call
  # format RGB input
  rgb_input = format_image(screenshot)

  # format language input
  tokenized_dom = tokenizer.tokenize_dom(new_dom)
  tokenized_dom = np.array(tokenizer.truncate_pad_entry(tokenized_dom, 492)) # 492 is the value set for CC-NeT5 processing
  tokenized_utterance = np.array(tokenizer.truncate_pad_entry(tokenizer.tokenize_string(utterance), 16))
  tokenized_task_name = np.array(tokenizer.truncate_pad_entry(tokenizer.tokenize_string(task_name), 4))
  rgb_data, language_input = format_language_rgb_input(rgb_input, tokenized_dom, tokenized_utterance, tokenized_task_name)
  # Unsqueeze with batch size to add first dimension
  rgb_data = rgb_data.unsqueeze(0)
  language_input = language_input.unsqueeze(0)

  # format previous action (flattened tensor size 577)
  previous_action = previous_action.unsqueeze(0)
  t5_output_tensor = t5_output_tensor.unsqueeze(0)

  # Inference to model CC-NeT5
  ccnet5_payload = prepare_ccnet5_payload(rgb_data, language_input, previous_action, t5_output_tensor)
  ccnet5_output = infer_remote_model(ccnet5_payload, cc_net5_url)


  # 5: Get ref output and pass into decoding dictionary
  # 6: Perform action
  time.sleep(2)       # Only here to let you look at the environment.

  # Find the HTML element with text "ONE".
  for element in observation["dom_elements"]:
    if element["text"] == "ONE":
      break

  # Click on the element.
  action = env.action_space.sample()     # Template for the action.
  action["action_type"] = env.action_space_config.action_types.index(
      ActionTypes.CLICK_ELEMENT
  )
  action["ref"] = element["ref"]
  observation, reward, terminated, truncated, info = env.step(action)


  # 7: Get reward
  # Check if the action was correct.
  assert reward >= 0      # Should be around 0.8 since 2 seconds has passed.
  assert terminated is True
  time.sleep(2)

  print(f'reward: {reward}')


  # 8: Save episode data with states


finally:
  env.close()


screenshot shape: (210, 160, 3)
t5_input: Click button ONE.<body ref=1 width=1200.0 height=210.0 bg_color=rgb(0.33, 0.33, 0.33) fg_color=rgb(0.0, 0.0, 0.0)><div ref=2 width=160.0 height=210.0 id=wrap bg_color=rgb(1.0, 1.0, 1.0) fg_color=rgb(0.0, 0.0, 0.0)><div ref=3 top=50.0 width=160.0 height=4.0 id=area bg_color=rgb(0.0, 0.0, 0.0) fg_color=rgb(0.0, 0.0, 0.0)><button ref=4 left=58.0 top=159.0 width=40.0 height=40.0 text=ONE id=subbtn bg_color=rgb(0.94, 0.94, 0.94) fg_color=rgb(0.0, 0.0, 0.0)></button><button ref=5 left=36.0 top=97.0 width=40.0 height=40.0 text=TWO id=subbtn2 bg_color=rgb(0.94, 0.94, 0.94) fg_color=rgb(0.0, 0.0, 0.0)></button></div></div></body>
about to make request: {'instruction_input': 'Click button ONE.<body ref=1 width=1200.0 height=210.0 bg_color=rgb(0.33, 0.33, 0.33) fg_color=rgb(0.0, 0.0, 0.0)><div ref=2 width=160.0 height=210.0 id=wrap bg_color=rgb(1.0, 1.0, 1.0) fg_color=rgb(0.0, 0.0, 0.0)><div ref=3 top=50.0 width=160.0 height=4.0 id=area bg_color=rgb(0.0, 

In [22]:
# Start a new episode.
observation, info = env.reset()

dict_keys(['utterance', 'dom_elements', 'screenshot', 'fields'])


In [24]:
print(observation['dom_elements'])
doms = convert_to_nested_dict(observation['dom_elements'])

[{'ref': 1, 'parent': 0, 'left': array([0.], dtype=float32), 'top': array([0.], dtype=float32), 'width': array([1200.], dtype=float32), 'height': array([210.], dtype=float32), 'tag': 'body', 'text': '', 'value': '', 'id': '', 'classes': '', 'bg_color': array([0.33333334, 0.33333334, 0.33333334, 1.        ], dtype=float32), 'fg_color': array([0., 0., 0., 1.], dtype=float32), 'flags': array([1, 0, 0, 0], dtype=int8)}, {'ref': 2, 'parent': 1, 'left': array([0.], dtype=float32), 'top': array([0.], dtype=float32), 'width': array([160.], dtype=float32), 'height': array([210.], dtype=float32), 'tag': 'div', 'text': '', 'value': '', 'id': 'wrap', 'classes': '', 'bg_color': array([1., 1., 1., 1.], dtype=float32), 'fg_color': array([0., 0., 0., 1.], dtype=float32), 'flags': array([0, 0, 0, 0], dtype=int8)}, {'ref': 3, 'parent': 2, 'left': array([0.], dtype=float32), 'top': array([50.], dtype=float32), 'width': array([160.], dtype=float32), 'height': array([4.], dtype=float32), 'tag': 'div', 'tex

In [99]:
import random

# find all refs of the dom dict
def find_all_refs(dom):
    refs = []
    if 'ref' in dom.keys():
        refs.append(dom['ref'])
    if 'children' in dom.keys():
        for child in dom['children']:
            f_ref = find_all_refs(child)
            refs += f_ref
    return list(set(refs))

# provide ref list and return dictionary with randomized values
# also return dict to reconstruct it all
def randomize_ref_dict(refs):
    entries = refs.copy()
    random.shuffle(entries)
    ref_dict = {}
    for i, k in enumerate(refs):
        ref_dict[k] = entries[i]

    ref2rand = ref_dict
    rand2ref = {}
    for k in ref2rand.keys():
        rand2ref[ref2rand[k]] = k
    return ref_dict, rand2ref

# Assigns the randomized value to each ref of the dom dict
def attribute_random_refs_dom(dom, refs):
    if 'ref' in dom.keys():
        dom['ref'] = refs[dom['ref']]
    if 'children' in dom.keys():
        new_children = []
        for child in dom['children']:
            new_child = attribute_random_refs_dom(child, refs)
            new_children.append(new_child)
        dom['children'] = new_children
    return dom

# Also need to Find if a new ref element has been added
ref_dict, rand2ref = randomize_ref_dict(find_all_refs(doms))
attribute_random_refs_dom(doms, ref_dict)

{'ref': 4,
 'left': 0.0,
 'top': 0.0,
 'width': 1200.0,
 'height': 210.0,
 'tag': 'body',
 'text': '',
 'value': '',
 'id': '',
 'classes': '',
 'bg_color': 'rgb(0.33, 0.33, 0.33)',
 'fg_color': 'rgb(0.0, 0.0, 0.0)',
 'children': [{'ref': 3,
   'left': 0.0,
   'top': 0.0,
   'width': 160.0,
   'height': 210.0,
   'tag': 'div',
   'text': '',
   'value': '',
   'id': 'wrap',
   'classes': '',
   'bg_color': 'rgb(1.0, 1.0, 1.0)',
   'fg_color': 'rgb(0.0, 0.0, 0.0)',
   'children': [{'ref': 5,
     'left': 0.0,
     'top': 50.0,
     'width': 160.0,
     'height': 4.0,
     'tag': 'div',
     'text': '',
     'value': '',
     'id': 'area',
     'classes': '',
     'bg_color': 'rgb(0.0, 0.0, 0.0)',
     'fg_color': 'rgb(0.0, 0.0, 0.0)',
     'children': [{'ref': 2,
       'left': 100.0,
       'top': 112.0,
       'width': 40.0,
       'height': 40.0,
       'tag': 'button',
       'text': 'ONE',
       'value': '',
       'id': 'subbtn',
       'classes': '',
       'bg_color': 'rgb(0.94

In [42]:
doms['children']

[{'ref': 2,
  'parent': 1,
  'left': array([0.], dtype=float32),
  'top': array([0.], dtype=float32),
  'width': array([160.], dtype=float32),
  'height': array([210.], dtype=float32),
  'tag': 'div',
  'text': '',
  'value': '',
  'id': 'wrap',
  'classes': '',
  'bg_color': array([1., 1., 1., 1.], dtype=float32),
  'fg_color': array([0., 0., 0., 1.], dtype=float32),
  'flags': array([0, 0, 0, 0], dtype=int8),
  'children': [{'ref': 3,
    'parent': 2,
    'left': array([0.], dtype=float32),
    'top': array([50.], dtype=float32),
    'width': array([160.], dtype=float32),
    'height': array([4.], dtype=float32),
    'tag': 'div',
    'text': '',
    'value': '',
    'id': 'area',
    'classes': '',
    'bg_color': array([0., 0., 0., 0.], dtype=float32),
    'fg_color': array([0., 0., 0., 1.], dtype=float32),
    'flags': array([0, 0, 0, 0], dtype=int8),
    'children': [{'ref': 4,
      'parent': 3,
      'left': array([81.], dtype=float32),
      'top': array([128.], dtype=float32)

In [232]:
task_name = 'click-checkboxes-transfer'
env = gymnasium.make(f"miniwob/{task_name}")

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/gymnasium/envs/registration.py:586: UserWarning: WARN: Using the latest versioned environment `miniwob/click-checkboxes-transfer-v1` instead of the unversioned environment `miniwob/click-checkboxes-transfer`.
  logger.warn(


In [251]:
# Define processing steps of a time step for the agent.
# Returns an action, and ref dictionaries over the steps.
# Pass ref dict of previous actions, empty for first step.
def observe_and_act(observation, ref_dict, rand2ref):
    #doms = observation['dom_elements']
    nested_dict = convert_to_nested_dict(observation['dom_elements'])
    _, _, new_dom = iterate_dom(nested_dict)
    ref_dict_new, _rand2ref_new = randomize_ref_dict(find_all_refs(new_dom))
    ref_dict, rand2ref = update_ref2_random(ref_dict, rand2ref, ref_dict_new)
    doms = attribute_random_refs_dom(new_dom, ref_dict)

    # Sample and Create Action
    action = env.action_space.sample()     # Template for the action.
    action["action_type"] = env.action_space_config.action_types.index(
      ActionTypes.CLICK_ELEMENT
    )
    # Get the ref number by using the dictionary
    action["ref"] = str(rand2ref[6])

    return action, ref_dict, rand2ref

# Empty ref dictionaries at start of episode:
ref_dict, rand2ref = {}, {}
observation, info = env.reset()

In [254]:
for i in range(1):
    print(f'Step {i}')
    # Process observation and return Action + Ref Dicts
    action, ref_dict, rand2ref = observe_and_act(observation, ref_dict, rand2ref)
    # Perform action on environment
    observation, reward, terminated, truncated, info = env.step(action)

    time.sleep(1)



Step 0


In [295]:
# Test loss serialization
import torch
import torch.nn as nn
import pickle

mse_loss_fn = nn.MSELoss()
mse_loss_value = mse_loss_fn(torch.randn(10), torch.randn(10))
#print(f'mse_loss_value: {mse_loss_value.item()}')
serialized_loss_value = pickle.dumps(mse_loss_value).decode('latin1')

# Create a dictionary payload
payload = {
    'loss_value': serialized_loss_value
}

# Convert the payload to JSON
json_payload = json.dumps(payload)

In [296]:
import requests
url_backprop_loss = 'https://ed9a-35-221-9-228.ngrok-free.app'

headers = {'Content-Type': 'application/json'}
response = requests.post(url_backprop_loss + '/backprop_actor_critic', data=json_payload, headers=headers)
print(response)

<Response [200]>


In [274]:
serialized_loss_value = json_payload['loss_value']

# Deserialize the loss function
loss_value = pickle.loads(serialized_loss_value.encode('latin1'))
print(f'got loss_value: {loss_value.item()}')


TypeError: string indices must be integers

In [297]:
rgb = torch.randn(3, 210, 160)
language = torch.randn(512)
old_previous_actions = torch.randn(577)
old_t5_out_tensors = torch.randn(512)


In [299]:
torch.stack((rgb, language))

RuntimeError: stack expects each tensor to be equal size, but got [3, 210, 160] at entry 0 and [512] at entry 1

In [306]:
cc_net5_url = 'https://1481-35-226-96-204.ngrok-free.app'
ccnet5_payload = prepare_ccnet5_payload(rgb, language, old_previous_actions, old_t5_out_tensors)
ccnet5_output_tensor = infer_remote_model(ccnet5_payload, cc_net5_url + '/infer_actor_normal')

about to make request: {'instruction_input': '{"rgb_input": [[[0.8501248955726624, 0.2356870323419571, 0.9747331142425537, -2.375483751296997, 0.31141233444213867, -0.9967247843742371, -0.8727461099624634, 0.509743869304657, 0.539456844329834, 1.2398244142532349, 1.1281205415725708, -2.438812017440796, -0.48125624656677246, 0.5461412668228149, 0.32093876600265503, -0.7621840238571167, 0.154116690158844, -0.315664678812027, -0.3113425076007843, -1.9390143156051636, 1.670494794845581, 0.09639942646026611, -0.7937015891075134, 0.21743564307689667, -0.649680495262146, -1.9340823888778687, 0.24054725468158722, -0.16791938245296478, -0.15238767862319946, 0.8637579679489136, 1.2145614624023438, 0.455335795879364, -0.6112703084945679, -0.2796092629432678, -1.1975122690200806, -1.2856676578521729, 0.0397387370467186, -0.558559000492096, 0.9652101993560791, -1.6201773881912231, 1.2448550462722778, -0.5935295224189758, -1.4814156293869019, -0.7381502389907837, 0.8159345388412476, 0.43395742774009

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [309]:
torch.randn(1, 577).squeeze().shape

torch.Size([577])

In [6]:
import torch
import torch.nn as nn

# Embedding Function
class EmbeddingFunction(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(EmbeddingFunction, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

    def forward(self, input_tokens):
        embedded = self.embedding(input_tokens)
        return embedded

vocab_size = 1000  # Number of unique tokens in the vocabulary
embedding_dim = 100  # Dimensionality of the embedding
embedding_func = EmbeddingFunction(vocab_size, embedding_dim)

# Generate some random input tokens and target tensor
batch_size = 32
sequence_length = 10
input_tokens = torch.randn(batch_size, sequence_length).long()
target = torch.randn(batch_size, sequence_length).long()

# Forward pass to get the output embedding
output_embedding = embedding_func(input_tokens)

# Reshape the output_embedding tensor
output_embedding = output_embedding.view(-1, embedding_dim)

# Compute the cross-entropy loss
loss_fn = nn.CrossEntropyLoss()
loss = loss_fn(output_embedding, target.view(-1))

IndexError: index out of range in self

In [7]:
import pandas as pd
df = pd.read_csv('/Users/lucas-andreithil/PycharmProjects/miniwob-plusplus/successful_episodes_df.csv')
#df.values.tolist()

In [8]:
df.head()

,task_name,utterance,states,reward
0,click-test-transfer,Click button ONE.,"{'successful_episodes_dom': [{'ref': 4, 'tag':...",0.159367
1,click-test-transfer,Click button ONE.,"{'successful_episodes_dom': [{'ref': 4, 'tag':...",0.853800


In [9]:
print(len(df))

2
